# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [23]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pprint import pprint

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,invercargill,-46.4000,168.3500,11.68,61,70,5.81,NZ,1731271665
1,1,novodvinsk,64.4165,40.8122,2.86,74,100,2.72,RU,1731271774
2,2,whitehorse,60.7161,-135.0538,0.01,74,75,5.66,CA,1731271779
3,3,shihezi,44.3000,86.0333,3.41,70,1,1.30,CN,1731271780
4,4,iqaluit,63.7506,-68.5145,-3.15,63,75,5.14,CA,1731271561


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = .75,
    color = "City",
    alpha = .7
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
max_temp = 28
min_temp = 20
wind_speed = 4
cloudiness = 0

ideal_df = city_data_df[
    (city_data_df["Max Temp"] <= max_temp) &
    (city_data_df["Max Temp"] >= min_temp) &
    (city_data_df["Wind Speed"] <= wind_speed) &
    (city_data_df["Cloudiness"] == cloudiness)
]

# Drop any rows with null values
ideal_df = ideal_df.dropna()

# Display sample data
ideal_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
40,40,crane,31.3974,-102.3501,23.76,12,0,3.52,US,1731271820
42,42,howrah,22.5892,88.3103,24.00,78,0,2.06,IN,1731271822
75,75,saint-pierre,-21.3393,55.4781,23.82,73,0,2.06,RE,1731271859
97,97,saucillo,28.0167,-105.2833,26.50,12,0,0.94,MX,1731271883
133,133,marrakesh,31.6315,-8.0083,22.04,43,0,3.60,MA,1731271922
165,165,al jawf,29.5000,38.7500,21.65,36,0,3.94,SA,1731271958
166,166,pamandzi,-12.7967,45.2794,25.88,83,0,1.54,YT,1731271959
183,183,cruzeiro do sul,-29.5128,-51.9853,26.39,54,0,3.00,BR,1731271977
203,203,zouerate,22.7187,-12.4521,25.11,16,0,2.78,MR,1731271999
243,243,al hazm,16.1641,44.7769,21.23,33,0,1.10,YE,1731272047


### Step 3: Create a new DataFrame called `hotel_df`.

In [13]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
40,crane,US,31.3974,-102.3501,12,
42,howrah,IN,22.5892,88.3103,78,
75,saint-pierre,RE,-21.3393,55.4781,73,
97,saucillo,MX,28.0167,-105.2833,12,
133,marrakesh,MA,31.6315,-8.0083,43,
165,al jawf,SA,29.5000,38.7500,36,
166,pamandzi,YT,-12.7967,45.2794,83,
183,cruzeiro do sul,BR,-29.5128,-51.9853,54,
203,zouerate,MR,22.7187,-12.4521,16,
243,al hazm,YE,16.1641,44.7769,33,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [35]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "filter":"",
    "limit":20,
    "bias":"",
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
crane - nearest hotel: No hotel found
howrah - nearest hotel: Sun India Guest House
saint-pierre - nearest hotel: Tropic Hotel
saucillo - nearest hotel: No hotel found
marrakesh - nearest hotel: No hotel found
al jawf - nearest hotel: No hotel found
pamandzi - nearest hotel: Gite Le Madina
cruzeiro do sul - nearest hotel: Estrela Palace Hotel
zouerate - nearest hotel: فندق تيرس زمور
al hazm - nearest hotel: No hotel found
pacific grove - nearest hotel: Pacific Grove Inn
laguna - nearest hotel: Holiday Inn Express & Suites
tazacorte - nearest hotel: App Leyma
al majaridah - nearest hotel: No hotel found
kill devil hills - nearest hotel: Mariner Days Inn & Suites
los llanos de aridane - nearest hotel: Valle Aridane
sur - nearest hotel: Sur Hotel
sao vicente - nearest hotel: Pousada Vitória
gilroy - nearest hotel: Days Inn by Wyndham Gilroy
jose pedro varela - nearest hotel: No hotel found
maintirano - nearest hotel: Hotel Safari
qaisumah - nearest hotel: No hotel fo

,City,Country,Lat,Lng,Humidity,Hotel Name
40,crane,US,31.3974,-102.3501,12,No hotel found
42,howrah,IN,22.5892,88.3103,78,Sun India Guest House
75,saint-pierre,RE,-21.3393,55.4781,73,Tropic Hotel
97,saucillo,MX,28.0167,-105.2833,12,No hotel found
133,marrakesh,MA,31.6315,-8.0083,43,No hotel found
165,al jawf,SA,29.5000,38.7500,36,No hotel found
166,pamandzi,YT,-12.7967,45.2794,83,Gite Le Madina
183,cruzeiro do sul,BR,-29.5128,-51.9853,54,Estrela Palace Hotel
203,zouerate,MR,22.7187,-12.4521,16,فندق تيرس زمور
243,al hazm,YE,16.1641,44.7769,33,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [41]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = .75,
    color = "City",
    alpha = .7,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)